In [1]:
import requests
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from collections import Counter

#워드클라우드 다운 링크 : https://www.lfd.uci.edu/~gohlke/pythonlibs/#wordcloud

In [2]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Hannanum
from konlpy.tag import Okt

In [3]:
# 참고 사이트 https://yeo0.github.io/data/2018/09/24/5.-%EB%A1%9C%EA%B7%B8%EC%9D%B8%EC%9D%B4-%ED%95%84%EC%9A%94%ED%95%9C-%EC%82%AC%EC%9D%B4%ED%8A%B8%EC%97%90%EC%84%9C%EC%9D%98-%ED%81%AC%EB%A1%A4%EB%A7%81/
# 참고 사이트 2 https://hashcode.co.kr/questions/9084/%EC%9E%A1%ED%94%8C%EB%9E%98%EB%8B%9B-%EB%A1%9C%EA%B7%B8%EC%9D%B8-%ED%9B%84-%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91-%EB%AC%B8%EC%9D%98%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4-%E3%85%9C%E3%85%9C

# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

In [4]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),5)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [5]:
# 각 산업군의 기업 리뷰 스크래핑
reviews1 = dict(ind_reviews(1001))

In [6]:
reviews1

{'전반적으로 만족스러운 직장생활. 안정적인 업무환경과 좋은 동료직원들': '86',
 '좋은 사람들과 좋은 환경\r지금은 어떨지 모르겠지만 이때가 가장 좋았던 것 같다.': '86',
 '워라밸을 중시한다면 최고의 직장입니다.': '86',
 '기회만 있다면 다시 일하고 싶은곳. 사람들이 정말 좋았다. 소방행정에 관심이 있다면 매우 도움이 될 것.': '86',
 '정말 좋은 사람들이랑 같이 일 할 수 있는 곳. 평생교육사로써 본인의 역량을 마음껏 발휘 할 수 있는 곳.': '86',
 '공무원집단에서 계약직으로 일하는게 이방인 느낌이 나기도 하지만 구성원들이 대부분 매너있다.': '86',
 '사서로서 일하기 좋은 곳이라고 생각합니다. 친절하신 분들이 많고 업무도 체계적으로 구성되어 진행되고 있었어요.': '86',
 '급여가 생가보다 쌤, 체계가 생각보다 없음, 커리어보단 돈': '86',
 '선거철 아르바이트로 하기에 좋은 일자리': '86',
 '타 관공서와 비교했을 때 업무가 독립적인면이있어 1년마다 자리가 바뀌는 업무상 어려움과 업무량의 업다운에도 불구하고 내일만 잘하면되는면이있음 모든 법원이 다 마찬가지임': '86',
 '대체로 여유로운 분위기 그러나 사람마다 업무 강도 차이가 커보얐음': '78',
 '넓은 캠퍼스에서 일할 수 있고 대학교 시설을 자유롭게 이용할 수 있음. 대신, 다른 지자체에 비해 체계가 없는 편이라고 함.': '78',
 '워라벨이 잘지켜지고, 기업문화가 상당히 수평적이라고  할 수 있다.': '78',
 '워라밸이 좋음 \r칼퇴 보장 계약직에 한해서는': '78',
 '보람차게 일할 수 있는 곳. 지식을 얻고자 오는 분들께 서비스를 제공할 수 있다는게 보람찼음.': '78',
 '그냥 저냥 공무윈입니다 다른 자치구에 비해 그닥 복지가 좋은 쪽은 아닌듯': '78',
 '생각보다 보수적인 것 같지않아요 사람들도 좋은 것같아요': '78',
 '일과 삶의 균형이 그 어느 곳보다 뛰어난 직장': '78',
 '당신이 국회 공무

In [7]:
#특수문자 제거
def sub(dic):
    dic = [re.sub(r"[^가-힣A-Za-z0-9]", " ", str(content)) for content in dic]
    return dic

In [8]:
#산업군 별 키워드 추출 후 리스트 생성
def extend(text):
    kkma=Kkma()
    temp=[]
    for i in text:
        temp.extend(kkma.nouns(i))
    return temp

In [9]:
review=extend(sub(reviews1))
review

['전반적',
 '만족',
 '직장',
 '직장생활',
 '생활',
 '안정적',
 '업무',
 '업무환경',
 '환경',
 '동료',
 '동료직원',
 '직원',
 '사람',
 '환경',
 '지금',
 '이때',
 '워라밸',
 '중시',
 '최고',
 '직장',
 '사람',
 '일',
 '수',
 '곳',
 '평생',
 '평생교육사',
 '교육사',
 '본인',
 '역량',
 '발휘',
 '공무원',
 '공무원집단',
 '집단',
 '계약',
 '계약직',
 '직',
 '이방',
 '느낌',
 '나기',
 '구성원',
 '대부분',
 '매너',
 '사서',
 '곳',
 '생각',
 '친절',
 '분',
 '업무',
 '체계적',
 '구성',
 '진행',
 '급여',
 '생가',
 '체계',
 '생각',
 '커리어',
 '돈',
 '선거철',
 '아르바이트',
 '일자리',
 '관공서',
 '비교',
 '때',
 '업무',
 '독립적',
 '면',
 '1',
 '1년',
 '년',
 '자리',
 '업무상',
 '어려움',
 '업무량',
 '업다운',
 '내일',
 '있음',
 '모든',
 '법원',
 '마찬가지',
 '마찬가지임',
 '임',
 '여유',
 '분위기',
 '사람',
 '업무',
 '강도',
 '차이',
 '보얐',
 '캠퍼스',
 '수',
 '대학교',
 '시설',
 '자유',
 '이용',
 '있음',
 '대신',
 '지자체',
 '체계',
 '편',
 '함',
 '워라벨',
 '기업',
 '기업문화',
 '문화',
 '수평적',
 '수',
 '워라밸',
 '칼',
 '칼퇴',
 '퇴',
 '보장',
 '계약',
 '계약직',
 '직',
 '수',
 '곳',
 '지식',
 '분',
 '서비스',
 '제공',
 '공무',
 '공무윈',
 '윈',
 '자치구',
 '복지',
 '쪽',
 '듯',
 '생각',
 '보수적',
 '사람',
 '일과',
 '삶',
 '균형',
 '곳',
 '직장',
 '당신',
 '국회',
 '공무원',
 '시험',
 

In [15]:
for i,v in enumerate(review):
    if len(v)<2:
        review.pop(i)

count=Counter(review)

words = dict(count.most_common())
words

{'업무': 10,
 '사람': 8,
 '기관': 7,
 '직장': 6,
 '공무원': 6,
 '생각': 6,
 '환경': 5,
 '분위기': 5,
 '경험': 5,
 '있음': 4,
 '생활': 3,
 '안정적': 3,
 '직원': 3,
 '집단': 3,
 '체계': 3,
 '강도': 3,
 '자유': 3,
 '근무': 3,
 '회사': 3,
 '공기업': 3,
 '공공': 3,
 '공공기관': 3,
 '만족': 2,
 '업무환경': 2,
 '워라밸': 2,
 '계약': 2,
 '계약직': 2,
 '느낌': 2,
 '친절': 2,
 '급여': 2,
 '일자리': 2,
 '1년': 2,
 '모든': 2,
 '기업': 2,
 '보장': 2,
 '제공': 2,
 '복지': 2,
 '보수적': 2,
 '그것': 2,
 '자기': 2,
 '업무강도': 2,
 '관심': 2,
 '가능': 2,
 '시스템': 2,
 '단점': 2,
 '희망': 2,
 '퇴근': 2,
 '꼰대': 2,
 '현장': 2,
 '전반적': 1,
 '직장생활': 1,
 '동료': 1,
 '동료직원': 1,
 '지금': 1,
 '이때': 1,
 '중시': 1,
 '최고': 1,
 '평생': 1,
 '평생교육사': 1,
 '교육사': 1,
 '본인': 1,
 '역량': 1,
 '발휘': 1,
 '공무원집단': 1,
 '이방': 1,
 '나기': 1,
 '구성원': 1,
 '대부분': 1,
 '매너': 1,
 '사서': 1,
 '체계적': 1,
 '구성': 1,
 '진행': 1,
 '생가': 1,
 '커리어': 1,
 '선거철': 1,
 '아르바이트': 1,
 '관공서': 1,
 '비교': 1,
 '독립적': 1,
 '자리': 1,
 '업무상': 1,
 '어려움': 1,
 '업무량': 1,
 '업다운': 1,
 '내일': 1,
 '법원': 1,
 '마찬가지': 1,
 '마찬가지임': 1,
 '여유': 1,
 '차이': 1,
 '보얐': 1,
 '캠퍼스': 1,
 '대학교': 1,
 '시설': 1,
 

In [61]:
!pip3 install WordCloud

  Using cached wordcloud-1.8.1.tar.gz (220 kB)
  Preparing metadata (setup.py) ... done
  ERROR: Command errored out with exit status 1:
   command: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/62/7f2mld7d4tzbtwwc3j2y4xk40000gp/T/pip-install-6fvpeuz3/wordcloud_729ccde054a54d44aede2baea439be9d/setup.py'"'"'; __file__='"'"'/private/var/folders/62/7f2mld7d4tzbtwwc3j2y4xk40000gp/T/pip-install-6fvpeuz3/wordcloud_729ccde054a54d44aede2baea439be9d/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/62/7f2mld7d4tzbtwwc3j2y4xk40000gp/T/pip-wheel-msrgq9hp
       cwd: /private/var/folders/62/7f2mld7d4tzbtwwc3j2y4xk40000gp/T/pip-install-6fv

In [16]:
from wordcloud import WordCloud 
import matplotlib.pyplot as plt 
%matplotlib inline 

import matplotlib 
from matplotlib import rc
rc('font', family='NanumBarunGothic')

wordcloud = WordCloud(
    font_path = '/Library/Fonts/NanumBarunGothic.ttf',  
    background_color='white',                             # 배경 색깔 정하기
    colormap = 'Accent_r',                                # 폰트 색깔 정하기
    width = 800,
    height = 800
)

wordcloud_words = wordcloud.generate_from_frequencies(words)


array = wordcloud.to_array()
print(type(array)) # numpy.ndarray
print(array.shape) # (800, 800, 3)

fig = plt.figure(figsize=(10, 10))
plt.imshow(array, interpolation="bilinear")
plt.axis('off')
plt.show()
fig.savefig('business_anlytics_worldcloud.png')

ModuleNotFoundError: No module named 'wordcloud'